# RNN을 이용한 텍스트 생성
- '경마장에 있는 말이 뛰고 있다'
- '그의 말이 법이다'
- '가는 말이 고와야 오는 말이 곱다'

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [4]:
# 단어 집합 생성
t = Tokenizer()
t.fit_on_texts([text])

In [5]:
t.word_index

{'가는': 8,
 '경마장에': 2,
 '고와야': 9,
 '곱다': 11,
 '그의': 6,
 '뛰고': 4,
 '말이': 1,
 '법이다': 7,
 '오는': 10,
 '있는': 3,
 '있다': 5}

In [6]:
# 단어 집합 크기 설정
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기: %d' % vocab_size)

단어 집합의 크기: 12


In [7]:
t.texts_to_sequences(['그의 말이 법이다'])

[[6, 1, 7]]

In [8]:
sequences = []
for line in text.split('\n'):
  encoded = t.texts_to_sequences([line])[0]
  for i in range(1, len(encoded)):
    sequence = encoded[:i+1]
    sequences.append(sequence)

print('학습에 사용할 샘픔의 개수: %d' % len(sequences))

학습에 사용할 샘픔의 개수: 11


In [9]:
sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

In [10]:
# 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
max_len = max(len(s)for s in sequences)
print('샘플의 최대 길이: ', max_len)

샘플의 최대 길이:  6


In [11]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
sequences[:5]

array([[0, 0, 0, 0, 2, 3],
       [0, 0, 0, 2, 3, 1],
       [0, 0, 2, 3, 1, 4],
       [0, 2, 3, 1, 4, 5],
       [0, 0, 0, 0, 6, 1]], dtype=int32)

In [12]:
X = sequences[:, :-1]
y = sequences[:, -1]

Y = to_categorical(y, num_classes=vocab_size)
Y[:3]

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

### 모델 정의
- Embedding
- SimpleRNN


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [14]:
model = Sequential([
                    Embedding(vocab_size, 5, input_length=max_len - 1),
                    SimpleRNN(32),
                    Dense(vocab_size, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 5, 5)              60        
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                1216      
_________________________________________________________________
dense (Dense)                (None, 12)                396       
Total params: 1,672
Trainable params: 1,672
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
history = model.fit(X, Y, epochs=200, verbose=0)

In [17]:
history.history['accuracy'][-1]

0.9090909361839294

### 모델 검증

In [18]:
def sentence_generation(model, t, current_word, n):
  init_word = current_word
  sentence = ''
  for _ in range(n):
    encoded = t.texts_to_sequences([current_word])[0]
    encoded = pad_sequences([encoded], maxlen=5, padding='pre')
    result = np.argmax(model.predict(encoded), axis=-1)
    for word, index in t.word_index.items():
      if index == result:
        break
      current_word = current_word + ' ' + word
      sentence = sentence + ' ' + word
    
  sentence = init_word + sentence
  return sentence

In [19]:
sentence_generation(model, t, '경마장에', 3)

'경마장에'

In [20]:
sentence_generation(model, t, '그의', 2)

'그의'

In [21]:
sentence_generation(model, t, '가는', 5)

'가는'

### 모델 변화
- Embedding Vector 개수: [2, 4, 6, 8]
- RNN 노드 개수: [24, 32]

In [23]:
def change_model(n_embed, n_rnn):
  model = Sequential([
                      Embedding(vocab_size, n_embed, input_length=max_len - 1),
                      SimpleRNN(n_rnn),
                      Dense(vocab_size, activation='softmax')
  ])
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  history = model.fit(X, Y, epochs=200, verbose=0)
  print(f"Accuracy: {history.history['accuracy'][-1]:.4f}")

  print(sentence_generation(model, t, '경마장에', 3))
  print(sentence_generation(model, t, '그의', 2))
  print(sentence_generation(model, t, '가는', 5))

In [26]:
for n_embed in [2, 4, 6, 8]:
  for n_rnn in [24, 32]:
    print('===========================')
    print(f'n_embed={n_embed}, n_rnn={n_rnn}')
    change_model(n_embed, n_rnn)
    print()

n_embed=2, n_rnn=24
Accuracy: 0.9091
경마장에
그의
가는

n_embed=2, n_rnn=32
Accuracy: 0.9091
경마장에
그의
가는

n_embed=4, n_rnn=24
Accuracy: 0.9091
경마장에
그의
가는

n_embed=4, n_rnn=32
Accuracy: 0.9091
경마장에 말이 경마장에 말이 경마장에 있는 말이 경마장에 있는 뛰고
그의
가는

n_embed=6, n_rnn=24
Accuracy: 0.9091
경마장에 말이 경마장에 말이 경마장에 있는 말이 경마장에 있는 뛰고 있다 그의 법이다 가는 고와야
그의
가는

n_embed=6, n_rnn=32
Accuracy: 1.0000
경마장에 말이 경마장에 말이 경마장에 있는 뛰고 있다 그의 법이다 가는 고와야
그의
가는

n_embed=8, n_rnn=24
Accuracy: 0.9091
경마장에
그의
가는

n_embed=8, n_rnn=32
Accuracy: 1.0000
경마장에 말이 경마장에 말이 경마장에 있는
그의
가는

